In [3]:
from pandasql import sqldf  # SQL 실행에 필요
import pandas as pd 
import json

In [14]:
# open json file 
with open('../data/tabmwp/tabmwp_test.json', 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# json data to pandas dataframe
row = {}
cnt = 9
for idx, data in json_data.items():
    if cnt>0:
        cnt -= 1
        continue
    row = data
    row['id'] = idx
    break

In [15]:
print(row)

{'question': 'Look at the table. Then answer the question. At a price of $155, is there a shortage or a surplus?', 'choices': ['shortage', 'surplus'], 'answer': 'shortage', 'unit': None, 'table_title': None, 'table': 'Price | Quantity demanded | Quantity supplied\n$155 | 22,600 | 5,800\n$275 | 20,500 | 9,400\n$395 | 18,400 | 13,000\n$515 | 16,300 | 16,600\n$635 | 14,200 | 20,200', 'table_for_pd': {'Price': ['$155', '$275', '$395', '$515', '$635'], 'Quantity demanded': ['22,600', '20,500', '18,400', '16,300', '14,200'], 'Quantity supplied': ['5,800', '9,400', '13,000', '16,600', '20,200']}, 'row_num': 6, 'column_num': 3, 'solution': 'At the price of $155, the quantity demanded is greater than the quantity supplied. There is not enough of the good or service for sale at that price. So, there is a shortage.\n', 'ques_type': 'multi_choice', 'ans_type': 'boolean_text', 'grade': 5, 'split': 'test', 'id': '54'}


In [28]:
def execute_sql(table, sql):
    try:
        my_table = table
        globals()['my_table'] = table
        result = sqldf(sql, globals()).values.tolist()
        # 2D list -> 1D list로 펼치기
        # 예: [[ 'michael schumacher'], ['lewis hamilton']] -> ['michael schumacher','lewis hamilton']
        flattened = [str(item).strip().lower() for row in result for item in row]
        # 쉼표로 join
        result_str = ', '.join(flattened)
    except Exception as e:
        # 실행 실패 시, 빈 결과
        print(f"SQL 실행 실패: {e}")
        result_str = ""
    return result_str

In [29]:
table = pd.DataFrame(row['table_for_pd'])
query = row['question']
sql = '''
SELECT CASE WHEN quantity_demanded > quantity_supplied THEN 'Shortage' ELSE 'Surplus' END FROM my_table WHERE price = 155
'''
print('table:\n', table)
print('query:', query)
result = execute_sql(table, sql)
print('result:\n', result)

table:
   Price Quantity demanded Quantity supplied
0  $155            22,600             5,800
1  $275            20,500             9,400
2  $395            18,400            13,000
3  $515            16,300            16,600
4  $635            14,200            20,200
query: Look at the table. Then answer the question. At a price of $155, is there a shortage or a surplus?
SQL 실행 실패: (sqlite3.OperationalError) no such column: quantity_demanded
[SQL: 
SELECT CASE WHEN quantity_demanded > quantity_supplied THEN 'Shortage' ELSE 'Surplus' END FROM my_table WHERE price = 155
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
result:
 
